In [4]:
import os, sys, pickle
import numpy as np 
import pandas as pd 

# shap
import shap

# Neighbor
from sklearn.manifold import MDS

# Directories
data_dir = os.getcwd() + '/data'
code_dir = os.getcwd() + '/fpdash'
sys.path.append(code_dir)

In [5]:
# Import classifier
with open(os.getcwd() + '/data/clf.pickle', 'rb') as f:
    clf = pickle.load(f)
# Import NN
with open(os.getcwd() + '/data/nn.pickle', 'rb') as f:
    nn = pickle.load(f)

In [6]:
# Import classifier
with open(os.getcwd() + '/data/clf.pickle', 'rb') as f:
    clf = pickle.load(f)
# Import NN
with open(os.getcwd() + '/data/nn.pickle', 'rb') as f:
    nn = pickle.load(f)

# load case base data
X_base = pd.read_csv(os.getcwd() + '/data/X_base.csv')
X_base_decoded = pd.read_csv(os.getcwd() + '/data/X_base_decoded.csv')
meta_base = pd.read_csv(os.getcwd() + '/data/meta_base.csv')
SHAP_base = pd.read_csv(os.getcwd() + '/data/SHAP_base.csv')

# load alert data
X_alert = pd.read_csv(os.getcwd() + '/data/X_alert.csv')
X_alert_decoded = pd.read_csv(os.getcwd() + '/data/X_alert_decoded.csv')
meta_alert = pd.read_csv(os.getcwd() + '/data/meta_alert.csv')
SHAP_alert = pd.read_csv(os.getcwd() + '/data/SHAP_alert.csv')

In [7]:
def retrieve_neighbors(i, n_neighbors = 10):
    distances, neighbors = nn.kneighbors(SHAP_alert.iloc[[i]], n_neighbors=n_neighbors)
    return distances[0], neighbors[0]

def compute_mds(i, neighbors, space):
    """Compute x and y for multi-dimensional scaling plot.
    
    Parameters
    ----------
    i : int
        index of instance in X_test
    neighbors : np array [n_neighbors]
        array with indices of neighbors in X_train
    space : str, one from ['shap', 'feature']
        distances computed based on shap value space or feature value space
    """
    if space == 'shap':
        alert = SHAP_alert
        base = SHAP_base
    elif space == 'feature':
        alert = X_alert
        base = X_base
    else:
        raise ValueError("space not in ['shap', 'feature']")
    mds_values = np.vstack((np.array(alert.iloc[i]), np.array(base.iloc[neighbors])))
    mds = MDS(random_state=1, dissimilarity ='euclidean', metric=True)
    mds.fit(mds_values.astype(np.float64))
    x, y = mds.embedding_.transpose()
    return x, y

In [377]:
import plotly.graph_objs as go
import matplotlib 


"""
Colors
"""
# Spectral colormap
spectral_cmap = matplotlib.cm.get_cmap('Spectral')
spectral_rgb = []
norm = matplotlib.colors.Normalize(vmin=0, vmax=255)
for i in range(0, 255):
    k = matplotlib.colors.colorConverter.to_rgb(spectral_cmap(norm(i)))
    spectral_rgb.append(k)
spectral = []

n_entries = 255
for k in [x / n_entries for x in range(0, n_entries+1, 1)]:
    C = spectral_rgb[int(np.round(255*k))-1]
    spectral.append([k, 'rgb'+str((C[0], C[1], C[2]))])
    
# Border colors
opacity = 0.5
cat_colors = {'TP' : 'rgba(159, 211, 86, %s)' % opacity,
             'TN' : 'rgba(13, 181, 230, %s)' % opacity,
             'FP' : 'rgba(177, 15, 46, %s)' % opacity,
             'FN' : 'rgba(255, 165, 76, %s)' % opacity}


def scatter_neighbors(x, y, neighbors, meta_base):
    global spectral
    global cat_colors
    
    """
    PREP
    """
    # Retrieve meta information
    meta_neighbors = pd.DataFrame({'x' : x[1:], 'y' : y[1:], 
                  'performance' : meta_base['performance'].iloc[neighbors], 
                  'score' : meta_base['score'].iloc[neighbors], 
                  'index' : neighbors})
    """
    ADD TRACES
    """
    traces = []
    # Add neighbors
    for perf in ['TP', 'TN', 'FP', 'FN']:
        group = meta_neighbors[meta_neighbors['performance'] == perf]
        scatter = go.Scatter(
            x = group['x'],
            y = group['y'],
            mode = 'markers',
            marker = {'line' : {'width' : 0, 'color' : cat_colors[perf]},
                      'color' : group['score'],
                      'colorscale' : spectral,
                      'cmin' : 0,
                      'cmax' : 1,
                      'size' : 10},
            showlegend = False,
            name=perf,
            hoverinfo = 'text',
            hoveron = 'points',
            text = group['score'])
        traces.append(scatter)
    #Add alert
    traces.append(go.Scatter(
        x = [x[0]],
        y = [y[0]],
        mode = 'markers',
        marker = {'line' : {'width' : 3, 'color' : 'rgba(50, 50, 50, 1)'},
                  'size' : 15,
                  'color' : 'rgba(255, 255, 0, 0.5)'},
        name = 'Current alert',
        showlegend = True,
        hoverinfo = 'name',
        hoveron = 'points',
        text = 'Current alert'))
    # Add dummy colorbar
    traces.append(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(
            colorscale=spectral, 
            showscale=True,
            cmin=0,
            cmax=1,
            colorbar=dict(thickness=10, outlinewidth=0, title="""Prediction""")),
        showlegend = False,
        hoverinfo='none'))

    """
    Define layout
    """
    xaxis = {'fixedrange' : False,
             'showgrid' : True,
            'zeroline' : False,
            'showline' : False,
            'showticklabels' : False,
        }
    yaxis = {'fixedrange' : False,
            'showgrid' : True,
            'zeroline' : False,
            'showline' : False,
            'showticklabels' : False
        }
    margin = go.layout.Margin(l=0, r=0, t=0, b=0, pad=0)
    layout = go.Layout(yaxis = yaxis, xaxis = xaxis, margin = margin, height = 300,
                       hovermode = 'closest', legend = dict(y=-0.05, orientation='h'))
    fig = dict(data=traces, layout=layout)
    return fig

def update_performance(fig, border_width=4):
    global spectral
    if fig['data'][0]['marker']['line']['width'] == 0:
        fig['data'][5]['marker']['showscale'] = False
        for i in range(4):
            fig['data'][i]['marker']['line']['width'] = border_width
            fig['data'][i]['marker']['colorscale'] = [[0,'rgba(75, 75, 75, 1)'], [1, 'rgba(75, 75, 75, 1)']]
            fig['data'][i]['showlegend'] = True
            
    else:
        fig['data'][5]['marker']['showscale'] = True
        for i in range(4):
            fig['data'][i]['marker']['line']['width'] = 0
            fig['data'][i]['marker']['colorscale'] = spectral
            fig['data'][i]['showlegend'] = False
    return fig

In [378]:
i=1
n_neighbors=50
space = 'shap'
distances, neighbors = retrieve_neighbors(i, n_neighbors)
x, y = compute_mds(i, neighbors, space=space)

In [382]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

fig = scatter_neighbors(x, y, neighbors, meta_base)
fig = update_performance(fig)
fig = update_performance(fig)
plot(fig, config = {'displayModeBar' : False, 'showLink' : False})

'file:///Users/hildeweerts/Projects/fpdash/temp-plot.html'

In [363]:
fig['data'][3]

Scatter({
    'hoverinfo': 'text',
    'hoveron': 'points',
    'marker': {'cmax': 1,
               'cmin': 0,
               'color': array([0.42, 0.42, 0.43]),
               'line': {'color': 'rgba(255, 165, 76, 0.5)', 'width': 3},
               'size': 10},
    'mode': 'markers',
    'name': 'FN',
    'showlegend': True,
    'text': array([0.42, 0.42, 0.43]),
    'x': array([-0.02508251, -0.1403776 , -0.04551149]),
    'y': array([ 0.08475615, -0.01946036,  0.11462377])
})